In [33]:
%%javascript
require.config({
   paths: {cytoscape: 'http://localhost:8099/js/cytoscape-2.7.10'}
   })

<IPython.core.display.Javascript object>

In [34]:
# ensure the library is available
import requests
r = requests.get('http://localhost:8099/js/cytoscape-2.7.10.js')
assert(r.status_code == 200)

In [35]:
import re

In [36]:
import igraph as ig
DEF_SCALING = 100.0
def from_igraph(igraph_network, layout=None, scale=DEF_SCALING):
    new_graph = {}
    network_data = {}
    elements = {}
    nodes = []
    edges = []
    network_attr = igraph_network.attributes() # Convert network attributes
    for key in network_attr:
        network_data[key] = igraph_network[key]
    edges_original = igraph_network.es # get network as a list of edges
    nodes_original = igraph_network.vs
    node_attr = igraph_network.vs.attributes()
    for idx, node in enumerate(nodes_original):
        new_node = {}
        data = {}
        data['id'] = str(node.index)
        data['name'] = str(node.index)
        for key in node_attr:
            data[key] = node[key]
        new_node['data'] = data
        if layout is not None:
            position = {}
            position['x'] = layout[idx][0] * scale
            position['y'] = layout[idx][1] * scale
            new_node['position'] = position
        nodes.append(new_node)
    edge_attr = igraph_network.es.attributes()  # Add edges to the elements
    for edge in edges_original:
        new_edge = {}
        data = {}
        data['source'] = str(edge.source)
        data['target'] = str(edge.target)
        for key in edge_attr:
            data[key] = edge[key]
        new_edge['data'] = data
        edges.append(new_edge)
    elements['nodes'] = nodes
    elements['edges'] = edges
    new_graph['elements'] = elements
    new_graph['data'] = network_data
    return new_graph


def to_igraph(network):

    nodes = network['elements']['nodes']
    edges = network['elements']['edges']
    network_attr = network['data']

    node_count = len(nodes)
    edge_count = len(edges)

    g = ig.Graph()

    # Graph attributes
    for key in network_attr.keys():
        g[key] = network_attr[key]

    g.add_vertices(nodes)

    # Add node attributes
    node_attributes = {}
    node_id_dict = {}
    for i, node in enumerate(nodes):
        data = node['data']
        for key in data.keys():
            if key not in node_attributes:
                node_attributes[key] = [None] * node_count

            # Save index to map
            if key == 'id':
                node_id_dict[data[key]] = i

            node_attributes[key][i] = data[key]

    for key in node_attributes.keys():
        g.vs[key] = node_attributes[key]

    # Create edges
    edge_tuples = []
    edge_attributes = {}
    for i, edge in enumerate(edges):
        data = edge['data']
        source = data['source']
        target = data['target']
        edge_tuple = (node_id_dict[source], node_id_dict[target])
        edge_tuples.append(edge_tuple)
        for key in data.keys():
            if key not in edge_attributes:
                edge_attributes[key] = [None] * edge_count

            # Save index to map
            edge_attributes[key][i] = data[key]

    g.add_edges(edge_tuples)

    # Assign edge attributes
    for key in edge_attributes.keys():
        if key == 'source' or key == 'target':
            continue
        else:
            g.es[key] = edge_attributes[key]

    return g


In [37]:
import ipywidgets as widgets
import json
import time
import os.path
from IPython.display import display, HTML
from traitlets import Int, Unicode, Tuple, CInt, Dict, validate, observe

class SimpleCyjsPyWidget(widgets.DOMWidget):
    
    _view_name = Unicode('SimpleCyjsBrowserView').tag(sync=True)
    _view_module = Unicode('SimpleCyjsBrowserViewModule').tag(sync=True)
    frameHeight = Int(300).tag(sync=True)
    msgFromKernel = Unicode("{}").tag(sync=True)
    msgFromBrowser = Unicode("{}").tag(sync=True)
    _incomingMessage = {};
    status = "initial status message\n"
    _selectedNodes = [];
    
        
    def addGraph(self, g):
      self._resetMessage();
      gjson = from_igraph(g)
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});
     
    def addGraphWithLayout(self, g, layout):
      self._resetMessage();
      gjson = from_igraph(g, layout)
      self.msgFromKernel = json.dumps({"cmd": "addGraph", "status": "request",
                                       "callback": "",    "payload": gjson});
     
    def deleteGraph(self):
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "deleteGraph", "status": "request", "callback": "", "payload": ""});
     
    
    def setHeight(self, newHeight):
      self.frameHeight = newHeight
        
    def fitSelected(self, margin=50):
      self.status += "entering fitSelected (%d)\n" % margin
      self.msgFromKernel = json.dumps({"cmd": "fitSelected", "status": "request", "callback": "", "payload": margin});
        
    def fit(self, margin=50):
      self.status += "entering fit (%d)\n" % margin
      self.msgFromKernel = json.dumps({"cmd": "fit", "status": "request", "callback": "", "payload": margin});
        
    def getSelectedNodes(self):
       return(self._selectedNodes);

    def selectNodes(self, nodes):
      self.msgFromKernel = json.dumps({"cmd": "selectNodes", "status": "request",
                                       "callback": "", "payload": nodes});

    def _resetMessage(self):   # ensures that any ensuing message is seen as novel in the browser
       self.msgFromKernel = json.dumps({"cmd": "cleanSlate", "status": "nop", "callback": "", "payload": ""});


    def availableLayouts(self):
        return(["grid", "null", "random", "cose", "circle", "concentric", "breadthfirst"]);
    
    def setPosition(self, igraph_layout):   # the somewhat cryptic object created by igraph layouts
      tblPos = []
      for i in range(0, len(igraph_layout)):
         x = igraph_layout[i][0] * 100
         y = igraph_layout[i][1] * 100
         tblPos.append({"id": i, "x": x, "y": y})
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "setPosition", "status": "request", "callback": "", 
                                       "payload": tblPos});
       
    def loadStyleFile(self, filename):
      if(not os.path.isfile(filename)):
        print("file '%s' not found" % filename)
        return;
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "loadStyleFile", "status": "request", "callback": "", 
                                       "payload": filename});
        
    def layout(self, name):
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "layout", "status": "request", "callback": "", "payload": name});
        
        
    def getPositions(self):
      self._resetMessage();
      self.msgFromKernel = json.dumps({"cmd": "getPositions", "status": "request", "callback": "", "payload": ""});
        
    def clearSelection(self):
      self._resetMessage();
      self. msgFromKernel = json.dumps({"cmd": "clearSelection", "status": "request",
                                        "callback": "", "payload": ""});
        
    def setNodeAttributes(self, attributeName, nodeNames, values):
      self._resetMessage();
      self. msgFromKernel = json.dumps({"cmd": "setNodeAttributes", "status": "request",
                                        "callback": "", 
                                        "payload": {"attributeName": attributeName,
                                                    "nodeNames": nodeNames,
                                                    "values": values}})

    def setEdgeAttributes(self, attributeName, sourceNames, targetNames, edgeTypes, values):
        # g to be a class member variable?
        # nodeMap also, with vigilance for modification of underlying graph?
        # id lookup a member function?
      names = g.vs['name']
      ids = [g.vs.find(name).index for name in g.vs['name']]
      nodeMap = dict(zip(names, ids))
      sourceIDs = [nodeMap[name] for name in sourceNames]
      targetIDs = [nodeMap[name] for name in targetNames]
      self._resetMessage(); 
      self. msgFromKernel = json.dumps({"cmd": "setEdgeAttributes", "status": "request",
                                        "callback": "", 
                                        "payload": {"attributeName": attributeName,
                                                    "sourceNames": sourceIDs,
                                                    "targetNames": targetIDs,
                                                    "edgeTypes": edgeTypes,
                                                    "values": values}})

    @observe('msgFromBrowser')
    def msg_arrived(self, change):
       self.status += "msgFromBrowser has arrived: %f\n" % time.time()
       rawMessage = change['new']
       self._incomingMessage = json.loads(rawMessage)
       cmd = self._incomingMessage["cmd"]
       self.status += "msg: %s\n"  % cmd
       if(cmd == "updateSelectedNodes"):
          self._selectedNodes = self._incomingMessage["payload"]
         
    def getResponse(self):
       return(self._incomingMessage["payload"])

    def getFullResponse(self):
       return(self._incomingMessage)
        

In [38]:
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

In [39]:
widget = SimpleCyjsPyWidget()
display(widget)

In [40]:
# widget.getPositions()

In [41]:
# widget.msgFromBrowser

In [42]:
widget.msgFromBrowser

'{}'

In [43]:
# cy.setHeight(800)

In [44]:
from igraph import *
import random

networkFile = "toyNetwork2.tsv"
nodeAttributesFile = "type.noa"
lines = open(networkFile).readlines()
tbl_rel = []
for line in lines:
   tokens = line.rstrip("\n").split("\t")
   tokens = [x.strip() for x in tokens]
   if(len(tokens) == 3):
      tbl_rel.append(tokens)



In [45]:
lines = open(nodeAttributesFile).readlines()

nodeTypes=[]
for line in lines:
   (node, type) = line.rstrip("\n").split("\t")
   nodeTypes.append([node, type])

g = Graph(directed=True)
nodeNames = [x[0] for x in nodeTypes]
nodeTypeValues = [x[1] for x in nodeTypes]

g.add_vertices(nodeNames)
g.vs['type'] = nodeTypeValues


In [46]:
for row in tbl_rel:
  source = row[0];
  target = row[1];
  #print("adding edge, %s -> %s" % (source, target))
  g.add_edge(source, target)

g.es['edgeType'] = [x[2] for x in tbl_rel]
#g.es()['flux'] = random.sample(range(0, 1000), len(g.es()))


In [47]:
widget.deleteGraph()
widget.addGraph(g)

In [48]:
widget.loadStyleFile("style2.js")

In [49]:
widget.selectNodes("R3")
widget.fitSelected(200)

In [50]:
rtLayout = g.layout("rt")
widget.setPosition(rtLayout)
kkLayout = g.layout("kk")
widget.setPosition(kkLayout)
widget.fit(100)
#for i in range(len(rtLayout)):
#   print(rtLayout[i])


In [51]:
igraphLayoutStrategies = ["circle", "drl", "fruchterman_reingold", 
                          "fruchterman_reingold_3d", 
                          "kamada_kawai", "kamada_kawai_3d", "lgl",
                          "random", "random_3d", "reingold_tilford",
                          "reingold_tilford_circular", "sphere"]

In [52]:
#for strategy in igraphLayoutStrategies:
#   print("---- %s" % strategy)
#   layout = g.layout(strategy)
#   cy.setPosition(layout)
#   cy.fit(100)
#   time.sleep(1)

In [53]:
cy.getSelectedNodes()

NameError: name 'cy' is not defined

In [ ]:
cy.clearSelection()

In [ ]:
from pandas import *
tbl_flux = DataFrame.from_csv('flux.tsv', sep='\t', header=0)
tbl_flux

In [ ]:
reactionNames = tbl_flux.index.tolist()
conditionNames = tbl_flux.columns.tolist()

In [ ]:
values = tbl_flux[conditionNames[0]].tolist()
values

In [ ]:
cy.setNodeAttributes("flux", reactionNames, values)

In [ ]:
tbl_edgeFlux = read_csv('edgeFlux.tsv', sep='\t')
tbl_edgeFlux

In [ ]:
sourceNodes = tbl_edgeFlux["source"].tolist()
targetNodes = tbl_edgeFlux["target"].tolist()
edgeTypes = tbl_edgeFlux["edgeType"].tolist()
values = tbl_edgeFlux["cond3"]

In [ ]:
cy.setEdgeAttributes("flux", sourceNodes, targetNodes, edgeTypes, values)

In [ ]:
cy.selectNodes(["R3", "i"])

In [ ]:
cy.getSelectedNodes()

In [ ]:
cy.fitSelected(200)

In [ ]:
len(g.vs)

In [ ]:
g.vs[1]

In [ ]:
g.vs.find("R5").index

In [ ]:
names = g.vs['name']
ids = [g.vs.find(name).index for name in g.vs['name']]
nodeMap = dict(zip(names, ids))

In [ ]:
nodeMap["R3"]

In [ ]:
nodes = ["R1", "c", "R5", "g1"]
[nodeMap[name] for name in nodes]

In [ ]:
[g.vs.find(x).index for x in g.vs['name']]

In [ ]:
x